# Report

## Introduction and data

### Die Bedeutung von YouTube für Nachrichtenmedien

YouTube hat sich in den letzten Jahren zu einer der wichtigsten Plattformen für den Konsum von Videoinhalten entwickelt. Insbesondere für Nachrichtenmedien wie ntv ist die Plattform zu einem unverzichtbaren Kanal geworden, um ihre Zielgruppe zu erreichen. Die Analyse der Performance von YouTube-Videos und das Verständnis der Faktoren, die zum Erfolg einzelner Beiträge führen, sind dabei von entscheidender Bedeutung für die strategische Ausrichtung des Kanals.

Für Nachrichtenmedien stellt sich dabei die besondere Herausforderung, dass sie einerseits journalistischen Qualitätsstandards gerecht werden müssen, andererseits aber auch die Mechanismen der Plattform verstehen und nutzen müssen, um ihre Inhalte erfolgreich zu platzieren. Die vorliegende Analyse soll dazu beitragen, diese Balance besser zu verstehen und zu optimieren.

### Forschungsfrage und Hypothesen

Die zentrale Forschungsfrage dieser Analyse lautet: Welche Faktoren beeinflussen die Performance von Videos auf dem ntv YouTube-Kanal und wie können diese Erkenntnisse genutzt werden, um die Content-Strategie zu optimieren?

Dabei werden drei zentrale Hypothesen untersucht: Erstens wird angenommen, dass Videos zu bestimmten Themengebieten wie dem Ukrainekrieg im Durchschnitt mehr Aufrufe und eine längere Wiedergabedauer erzielen als Videos zu anderen Themen. Zweitens wird vermutet, dass die Art der Thumbnail-Gestaltung, insbesondere die prominente Platzierung bekannter Experten, einen signifikanten Einfluss auf die Klickrate hat. Die dritte Hypothese besagt, dass eine höhere Bewertung des Videotitels positiv mit der Anzahl der Aufrufe und der Wiedergabedauer korreliert.

### Datengrundlage und Erhebungsprozess

Der analysierte Datensatz umfasst etwa 2500 Videos, die im Jahr 2024 bis Mitte November auf dem ntv YouTube-Kanal veröffentlicht wurden. Die Datenerhebung gestaltete sich dabei als komplexer Prozess, der verschiedene Quellen und Methoden kombiniert. Während grundlegende Metriken wie Aufrufe, Likes und Kommentare über die YouTube Analytics API bezogen werden können, sind spezielle Metriken wie die Klickrate der Impressionen nur durch manuelle Extraktion aus dem YouTube Studio Backend verfügbar. Dies macht die Datensammlung zu einem aufwändigen Prozess, der sowohl automatisierte als auch manuelle Schritte erfordert.

Zusätzlich zu den von YouTube bereitgestellten Metriken wurden weitere Kategorisierungen und Bewertungen vorgenommen. Die Videos wurden nach ihren Themenbereichen kategorisiert (Politik, Wirtschaft, Krieg, Sonstiges, Bilder, Live). Die Thumbnail-Gestaltung wurde nach einem dreistufigen System bewertet, das insbesondere die Präsenz und Positionierung wichtiger Persönlichkeiten berücksichtigt. Die Kategorie "3" gab es bei Thumbnails die eine Kombination von Gesicht und Titel 

<img src="../references/images/kat3.jpg" width="300" alt="Kategorie 3 Visualisierung"/>

 besonders präsent ist. Die Kategorie "2" gab es, wenn eines von beiden vorhanden war:
 
<img src="../references/images/kat2.jpg" width="300" alt="Kategorie 3 Visualisierung"/>

  Entweder ein normales Bild + präsenten Tiel oder präsentes Gesicht. Eine "1" wurde für Thumbnails vergeben, die keine besonderen Merkmale aufgewiesen haben. 
  
  <img src="../references/images/kat1.jpg" width="300" alt="Kategorie 3 Visualisierung"/>
  


## Automatische Bewertung Thumbnails und Titel
  In der Datei "youtube Daten abrufen Test2.ipynb" ist auch ein Code abgelegt, der über "Google Vision AI" eine Bewertung der Thumnbails nach diesen Kategorien erfolgen soll. Leider war hier die Parameter zu ungenau. Eine Verbesserung des Systems wäre für das Projekt zu aufwendig und teuer gewesen. Hier müsste noch mehr Aufwand betrieben werden oder ein eigenes Modell trainiert werden. Das wäre die Aufgabe für ein neues Projekt.
  Für die Bewertung der Videotitel wurde ein KI-gestütztes System entwickelt, das die Titel nach SEO-Kriterien auf einer Skala von 0.0 bis 10,0 bewertet. Dabei wurden durch die KI Bewertungen zwischen 4,5 und 8,5 vorgenommen. Als KI wurde ChatGPT per API genutzt. Der Code und auch der Prompt dafür ist in der Datei youtube Daten abrufen Test2.ipynb abgelegt.  


In [ ]:
class YouTubeSEOScorer:
    def __init__(self, api_key: str, model: str = "gpt-3.5-turbo"):
        self.model = model
        self.client = OpenAI(api_key=api_key)
        self.checkpoint_manager = CheckpointManager()
        
        self.system_prompt = """Du bist ein YouTube SEO Experte, spezialisiert auf Nachrichtenkanäle. 
        Bewerte den YouTube-Titel nach folgenden Kriterien und vergib eine Gesamtpunktzahl von 0-10. 
        WICHTIG: Gib NUR die Gesamtpunktzahl als einzelne Zahl zurück, OHNE Text oder Erklärungen.
        Beispiele korrekter Antworten:
        7,5
        8,0
        6,5
        
        Bewerte nach diesen Kriterien:

        1. Titel-Optimierung (max. 2 Punkte)
        - Länge zwischen 40-70 Zeichen (+0.5)
        - Hauptkeyword in den ersten 3-4 Wörtern (+0.5)
        - Klare Struktur mit natürlicher Lesefluss (+0.5)
        - Verwendung von Trennzeichen (|, -, :) wo sinnvoll (+0.5)

        2. Nachrichtenwert & Aktualität (max. 2 Punkte)
        - Klare Vermittlung der Nachrichtenrelevanz (+0.5)
        - Zeitliche Einordnung wenn relevant (+0.5)
        - Balance zwischen Aktualität und Evergreen-Potenzial (+0.5)
        - Korrekte Priorisierung der Information (+0.5)

        3. Keyword-Optimierung (max. 2 Punkte)
        - Verwendung relevanter Nachrichtenkeywords (+0.5)
        - Natürliche Integration von Suchbegriffen (+0.5)
        - LSI Keywords / thematisch verwandte Begriffe (+0.5)
        - Vermeidung von Keyword-Stuffing (+0.5)

        4. Zielgruppen-Ansprache (max. 2 Punkte)
        - Verständliche Sprache für Nachrichtenpublikum (+0.5)
        - Seriosität und Professionalität (+0.5)
        - Klare Themenankündigung (+0.5)
        - Zielgruppengerechte Formulierung (+0.5)

        5. Technische Optimierung (max. 2 Punkte)
        - Keine übermäßige Großschreibung (+0.5)
        - Korrekte Zeichensetzung (+0.5)
        - Keine Spam-Taktiken oder Clickbait (+0.5)
        - Mobile-freundliche Länge & Format (+0.5)

        WICHTIG - FORMAT DER ANTWORT:
        - Gib ausschließlich eine einzelne Dezimalzahl zurück
        - Verwende ein Komma als Dezimaltrennzeichen
        - Keine Erklärungen oder zusätzlicher Text
        - Keine Aufschlüsselung der Einzelkriterien
        
        Beispiele korrekter Antworten:
        7,5
        8,0
        6,5"""


## Zentrale Metriken und ihre Bedeutung für einen Nachrichtenkanal

Für die Analyse wurden drei zentrale Zielvariablen identifiziert, die für Nachrichtenmedien auf YouTube von besonderer Bedeutung sind:

Die **durchschnittliche Wiedergabedauer** ist besonders für Nachrichtenformate relevant, da sie Aufschluss darüber gibt, wie gut es gelingt, komplexe Sachverhalte für die YouTube-Zielgruppe aufzubereiten. Anders als bei traditionellen TV-Nachrichten, wo Zuschauer tendenziell das gesamte Format konsumieren, ist die Haltedauer auf YouTube ein kritischer Erfolgsfaktor. Sie beeinflusst direkt, wie häufig ein Video vom Algorithmus anderen Nutzern vorgeschlagen wird. So zeigen erfolgreiche Videos des ntv-Kanals, dass besonders Live-Berichterstattungen zu wichtigen Ereignissen, wie beispielsweise die Iran-Angriffe auf Israel, überdurchschnittliche Wiedergabedauern erzielen. Bei diesen Events bleiben Zuschauer oft über 10 Minuten am Video, während der Durchschnitt bei Nachrichtenvideos bei etwa 4-5 Minuten liegt.

Die Anzahl der **Aufrufe** ist für Nachrichtenkanäle nicht nur eine reine Reichweitenkennzahl, sondern auch ein Indikator für die gesellschaftliche Relevanz ihrer Berichterstattung. Im hart umkämpften Nachrichtenmarkt auf YouTube, wo etablierte Medien mit einer Vielzahl alternativer Informationsquellen konkurrieren, sind hohe Aufrufzahlen zudem entscheidend für die Monetarisierung und damit die Finanzierung hochwertiger journalistischer Arbeit. Dabei spielen sowohl die Bekanntheit der Personen als auch die Aktualität und Relevanz der Themen eine entscheidende Rolle.

Die **Klickrate der Impressionen** hat für Nachrichtenmedien eine besondere strategische Bedeutung. Sie zeigt, wie gut es gelingt, in der Informationsflut von YouTube die Aufmerksamkeit potenzieller Zuschauer zu gewinnen, ohne dabei in reißerische oder irreführende Darstellungen zu verfallen. Erfolgreiche Beispiele aus dem ntv-Kanal demonstrieren, dass Videos mit klar erkennbaren Experten im Thumbnail und prägnanten, aber sachlichen Titeln Klickraten von über 10% erreichen können - deutlich über dem Durchschnitt von etwa 4-5%. Besonders erfolgreich sind dabei Thumbnails, die bekannte Persönlichkeiten in emotionalen Momenten oder bei markanten Aussagen zeigen, kombiniert mit kurzen, prägnanten Headlines.

***Limitationen und Herausforderungen***

Bei der Analyse müssen einige wichtige Einschränkungen berücksichtigt werden. Die teilweise notwendige manuelle Datenextraktion erhöht das Risiko von Fehlern und macht den Prozess zeitaufwändig. Die Kategorisierung der Themen und die Bewertung der Thumbnails enthalten zwangsläufig subjektive Elemente, auch wenn versucht wurde, diese durch klare Kriterien zu minimieren. 

## Kennzahlen und Ihre Bedeutung
Im YouTube-Universum sind die Begriffe „Impressions“, „Klickrate der Impressions“ und „Views“ zentrale Kennzahlen, die häufig für die Performance-Analyse eines Channels herangezogen werden. Hier eine kurze Erklärung:
### 1. Impressions (Sichtkontakte)

- **Was bedeutet das?**  
  Impressions geben an, wie oft ein Thumbnail (Vorschaubild) eines Videos auf YouTube den Nutzer*innen angezeigt wurde. Dabei spielt es keine Rolle, ob das Video letztendlich angeklickt und angesehen wird – jede einzelne Einblendung des Thumbnails in den Suchergebnissen, den Empfehlungen oder auf anderen YouTube-Oberflächen zählt als Impression.

- **Wozu dient diese Kennzahl?**  
  \- Sie zeigt das Potenzial für Reichweite: Wie viele Leute _könnten_ das Video sehen (und ggf. anklicken)?  
  \- Je höher die Impressions, desto mehr Nutzer*innen wurde das Video theoretisch präsentiert.

---

### 2. Klickrate der Impressions (Impression Click-Through-Rate, CTR)

- **Was bedeutet das?**  
  Die Klickrate (CTR) ist der prozentuale Anteil der Impressions, bei denen tatsächlich auf das Video geklickt wurde. Anders ausgedrückt: Wenn dein Thumbnail 100 Mal angezeigt wird und 10 Mal geklickt wird, beträgt die Klickrate 10 %.  

  \[
  \text{Klickrate der Impressions} = \frac{\text{Klicks auf das Video}}{\text{Impressions}} \times 100\% 
  \]

- **Wozu dient diese Kennzahl?**  
  \- Sie ist ein guter Indikator dafür, wie „anziehend“ der Titel und das Thumbnail deines Videos sind.  
  \- Eine niedrige Klickrate kann bedeuten, dass du das Video zwar oft ausgespielt bekommst (also viele Impressions), aber die Nutzer*innen sich nicht für den Klick entscheiden.  
  \- Eine höhere Klickrate deutet darauf hin, dass das Vorschaubild und der Titel des Videos besonders ansprechend sind und Nutzer*innen zum Anschauen animieren.

---

### 3. Views (Aufrufe)

- **Was bedeutet das?**  
  Views zählen, wie oft ein Video tatsächlich angesehen wurde – also wie oft jemand den Abspielknopf drückt (und das Video mindestens für eine gewisse Zeit ansieht). Die konkrete Definition, wann ein „View“ gezählt wird, hängt von YouTube-internen Kriterien ab (z. B. Mindestwiedergabedauer), sodass nicht sofort jeder Aufruf in Echtzeit als View erscheint.

- **Wozu dient diese Kennzahl?**  
  \- Views sind wohl das bekannteste Maß für den Erfolg eines Videos.  
  \- Zeigt dir, wie oft dein Content tatsächlich konsumiert wurde.  
  \- Zusammen mit weiteren Daten (z. B. Wiedergabedauer, Zuschauerbindung) bekommst du ein Bild davon, wie interessant die Inhalte sind und wie lange die Zuschauer*innen dabeibleiben.

---

### Wichtig zu wissen
- **Zusammenspiel der Kennzahlen**:  
  \- Hohe Impressions, aber niedrige Klickrate = Zeichen, dass das Video zwar vielen Nutzer*innen angezeigt wird, aber das Thumbnail/Titel nicht zum Klicken motiviert.  
  \- Niedrige Impressions, aber hohe Klickrate = Zeichen, dass das Video in der Zielgruppe, in der es ausgespielt wird, gut ankommt, aber insgesamt noch nicht genug Reichweite erzielt.  
  \- Views und Klickrate hängen indirekt zusammen. Eine höhere Klickrate führt in der Regel zu mehr Views – sofern YouTube das Video möglichst oft als Impression ausspielt.

- **Video-Optimierung**:  
  \- Experimentiere mit Thumbnails, Titel und Beschreibungen, um eine gute Balance zwischen **vielen Impressions** und einer **hohen Klickrate** zu erreichen.  
  \- Eine gute Zuschauerbindung (Retention) ist ebenfalls wichtig, weil YouTube-Algorithmen Videos mit hoher Zuschauerzufriedenheit häufiger vorschlagen.

- **Gerätenutzung**:    
  \- Für die Bewertung der Ergebnisse könnte auch wichtig sein, auf welchen Geräten die User die Youtube-Vides schauen:

    <span style="color: red; font-weight: bold;">  Handy: 61,9 %, Computer: 15,6 %, TV: 13,6 %, Tablet: 8,8 % </span>

  \- Oder das Geschlecht und Alter der User:

    <span style="color: red; font-weight: bold;">  Männlich 80%, Alter: 55+ 61 %, 25 bis 55: 35 % </span>

---

## Datenvorverarbeitung

Im Notebook **Datenaufbereitung** werden die RAW-Daten geladen und in verschiedenen Schritten aufbereitet. 

In [ ]:
#Imports
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScale
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

# Setzen der Display-Optionen für bessere Lesbarkeit
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

# Warnung unterdrücken
import warnings
warnings.filterwarnings('ignore')

# Plotstil festlegen (verwenden einen basic matplotlib style)
plt.style.use('default')
# Seaborn Plotting Style setzen
sns.set_theme(style="whitegrid")

## Schlussfolgerungen der bisherigen Datenanalyse und Datenvorverarbeitung


### **1. Datenstruktur und Vorbereitung**
- Der Datensatz enthält sowohl numerische als auch kategoriale Merkmale.
- Die Zielvariablen zeigen starke Rechtsschiefe mit vielen niedrigen und wenigen extrem hohen Werten.

<img src="../references/images/erste analyse der Zielvariablen.jpg" width="600" alt="Verteilung Zielvariablen"/>

- Merkmale wie **"video_id" wurden als Identifikationsspalten ausgeschlossen, da sie keinen direkten Einfluss auf die Vorhersage der Aufrufzahlen haben. 

### **2. Bewertung der Variablen**
- **Bewertung_Titel:**
  - Diese Variable wurde auf einer Skala von 0 bis 10 vergeben und sollte daher nicht skaliert werden, um die interpretierbare Einteilung (z. B. schlechte, durchschnittliche und gute Bewertungen) beizubehalten.
  - Die Verteilung der Werte zeigte eine starke Konzentration auf einige wenige Werte. Die reine SEO-Bewertung scheint nicht sinnvoll, da die KollegInnen bereits auf einem hohen Noveau arbeiten und die Bewertung zwischen 6 und 8 laufen. 

- **Gestaltung_Thumbnail:**
  - Diese Variable wurde in den Werten 1, 2 und 3 vergeben und spiegelt eine Einteilung in Kategorien wider (z. B. schlecht, neutral, gut).
  - Auch hier wurde festgestellt, dass eine Standardisierung nicht sinnvoll ist, da es sich um feste Kategorien handelt.

### **3. Zusammenhänge zwischen Prädiktoren und Zielvariable**
- **Themenzugehörigkeit:**
  - Das Thema `Krieg` zeigte einen deutlichen Einfluss auf die Aufrufzahlen. Videos mit diesem Thema hatten im Vergleich zu anderen Themen höhere Median-Aufrufzahlen und eine breite Streuung.
  - 
<img src="../references/images/boxplot Krieg_Aufrufe.JPG" width="400" alt="Boxplot Krieg vs. Aufrufe"/>

  - Im Gegensatz dazu zeigte das Thema `Politik` keinen signifikanten Unterschied zwischen Videos mit und ohne diese Zuordnung.

<img src="../references/images/boxplot Politik_Aufrufe.JPG" width="400" alt="Boxplot Politik vs. Aufrufe"/>

- **Bewertung_Titel und Gestaltung_Thumbnail vs. durchschn. Wiedergabedauer**
  - In den Streudiagrammen ein mäßiger Zusammenhang zwischen der Titelbewertung, den Thumnails mit der durchschnittlichen Wiedergabedauer festgestellt. Hier im Bild ist auch die Videolänge im Zusammenhang mit der durchschn. Wiedergabedauer zu sehen. Der Zusammenhang scheint jedoch logisch. Bis zu einem gewissen Grad sollte bei längeren Videos auch die Wiedergabedauer steigen. Dieser Zusammenhang dürfte irgendwo aber auch eine Peak haben. 
  
<img src="../references/images/streudiagramme_Wiedergabedauer_Thumbnail_Titel_Videolaenge.JPG" width="1000" alt="Boxplot Politik vs. Aufrufe"/>

### **Erste Zusammenfassung:**
Die bisherige Analyse hat gezeigt, dass die Merkmale im Datensatz sehr unterschiedlich strukturiert sind und nicht alle Prädiktoren gleich behandelt werden sollten. Durch eine gezielte Auswahl von Features und die Anwendung einer selektiven Skalierung kann das Modell stabilisiert werden. Zusätzliche Transformationen könnten die Analyseergebnisse verbessern und den Einfluss einzelner Prädiktoren verstärken. Die meisten Variablen werden in der Regressionsanalyse keine Rolle spielen, da sie nicht beeinflussbar sind, sondern Performance-Vaiablen darstellen. Diese sind sicherlich wichtig, um weitere Anaylsen zur Struktur der User und Verhalten durchzuführen.


#### **Ergebnis Vor-Analyse**

Die Entwicklung der Analysestrategie war ein iterativer Prozess, der durch mehrere wichtige Erkenntnisse geprägt wurde. Zu Beginn der Untersuchung wurden alle verfügbaren Prädiktoren einbezogen, wobei sich früh zeigte, dass die am stärksten beeinflussbaren Faktoren wie Thumbnail-Gestaltung und Titelbewertung überraschend geringe Korrelationen mit den Zielvariablen aufwiesen.

Eine genauere Untersuchung der Daten offenbarte, dass diese schwachen Zusammenhänge teilweise durch die besondere Charakteristik von Live-Übertragungen verzerrt wurden. Live-Videos folgen grundsätzlich anderen Gesetzmäßigkeiten als reguläre Nachrichtenvideos: Sie werden oft über Breaking News oder wichtige Ereignisse automatisch als Startseiten-Empfehlung ausgespielt, was zu überdurchschnittlich hohen Klickraten führt - unabhängig von Thumbnail oder Titel. Zudem werden Live-Videos häufig mit standardisierten Thumbnails und Titeln versehen, da bei Breaking News die Zeit für aufwändige Optimierungen fehlt.

**Warum Kategorie "Live" aus den Daten nehmen**

Diese Erkenntnis führte zu dem Entschluss, Live-Videos aus der Hauptanalyse auszuschließen, um die tatsächlichen Effekte der beeinflussbaren Faktoren bei regulären Nachrichtenvideos besser isolieren zu können. Die verbleibenden schwachen bis moderaten Zusammenhänge zwischen Thumbnail-Gestaltung, Titelbewertung und Performance-Metriken deuten darauf hin, dass der Erfolg von YouTube-Videos im Nachrichtenbereich von einem komplexen Zusammenspiel verschiedener Faktoren abhängt, die über rein gestalterische Aspekte hinausgehen.

**Ausreisseranalyse**

Die anschließende Ausreißeranalyse bestätigte diese Vermutung: Erst nach der Bereinigung extremer Ausreißer, die oft durch externe Faktoren wie besondere Nachrichtenereignisse oder algorithmusbedingte Empfehlungsschübe entstehen, wurden die tatsächlichen Effekte der Gestaltungselemente statistisch signifikant nachweisbar. Dies unterstreicht die Notwendigkeit, bei der Analyse von YouTube-Metriken im Nachrichtenbereich sowohl die besonderen Charakteristika verschiedener Videoformate als auch externe Einflussfaktoren zu berücksichtigen.

Diese Erkenntnisse führten zu einem verfeinerten Analyseansatz, der sich auf reguläre Nachrichtenvideos konzentriert und durch geeignete statistische Methoden wie die IQR-basierte Ausreißerbereinigung die subtileren Effekte der Gestaltungselemente besser herausarbeiten kann.

#### Ich habe die EDA durch folgenden Code gestaltet:

In [ ]:
import os
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

def remove_outliers(data, method='iqr', threshold=1.5):
    """
    Entfernt Ausreißer aus einer Serie basierend auf verschiedenen Methoden.
    
    Parameters:
    -----------
    data : pandas Series
        Zu bereinigende Daten
    method : str
        'iqr' für IQR-Methode
        'zscore' für Z-Score Methode
        'modified_zscore' für modifizierten Z-Score
    threshold : float
        Schwellenwert für die jeweilige Methode
    """
    if method == 'iqr':
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        return (data >= lower_bound) & (data <= upper_bound)
    
    elif method == 'zscore':
        z_scores = np.abs(stats.zscore(data))
        return z_scores < threshold
    
    elif method == 'modified_zscore':
        median = data.median()
        mad = stats.median_abs_deviation(data)
        modified_z_scores = 0.6745 * (data - median) / mad
        return np.abs(modified_z_scores) < threshold
    
    return pd.Series([True] * len(data))

def flexible_analysis(X_data, y_data, target_variable, predictor_variables, 
                     outlier_method=None, outlier_threshold=1.5, output_dir='analysis_output'):
    """
    Führt flexible Analyse mit optionaler Ausreißerbehandlung durch.
    
    Parameters:
    -----------
    X_data, y_data : DataFrames mit Prädiktoren und Zielvariablen
    target_variable : str, Name der Zielvariable
    predictor_variables : list, Liste der Prädiktorvariablen
    outlier_method : str oder None, Methode der Ausreißerbehandlung
    outlier_threshold : float, Schwellenwert für Ausreißerbehandlung
    output_dir : str, Ausgabeverzeichnis
    """
    
    # Erstelle Ausgabeverzeichnis falls nicht vorhanden
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Filtere Live-Videos
    #non_live_mask = X_data['theme_Live'] == 0
    #X_filtered = X_data[non_live_mask]
    #y_filtered = y_data[non_live_mask]
    X_filtered = X_data
    y_filtered = y_data

    # Kombiniere Daten für die Analyse
    analysis_data = pd.DataFrame()
    analysis_data[target_variable] = y_filtered[target_variable]
    for pred in predictor_variables:
        analysis_data[pred] = X_filtered[pred]
    
    # Ausreißerbehandlung wenn gewünscht
    if outlier_method:
        print(f"Ausreißerbehandlung mit Methode: {outlier_method}, Schwellenwert: {outlier_threshold}")
        initial_size = len(analysis_data)
        mask = remove_outliers(analysis_data[target_variable], method=outlier_method, threshold=outlier_threshold)
        analysis_data = analysis_data[mask]
        print(f"Entfernte Datenpunkte: {initial_size - len(analysis_data)} ({((initial_size - len(analysis_data))/initial_size*100):.1f}%)")
    
    # Grundlegende statistische Analyse
    print(f"\nAnalyse für Zielvariable: {target_variable}")
    print("="*50)
    
    print("\nDeskriptive Statistik für Zielvariable:")
    print(analysis_data[target_variable].describe())
    
    # Analyse je Prädiktor
    for pred in predictor_variables:
        print(f"\nAnalyse für Prädiktor: {pred}")
        print("-"*30)
        
        print("\nDeskriptive Statistik:")
        print(analysis_data[pred].describe())
        
        # Für kategorische und metrische Variablen
        is_categorical = analysis_data[pred].nunique() < 10
        if is_categorical:
            print("\nGruppierte Statistiken:")
            grouped_stats = analysis_data.groupby(pred)[target_variable].agg(['mean', 'std', 'count'])
            print(grouped_stats)
            
            # ANOVA
            groups = [group for _, group in analysis_data.groupby(pred)[target_variable]]
            if len(groups) >= 2:
                f_stat, p_val = stats.f_oneway(*groups)
                print(f"\nANOVA Test:")
                print(f"F-Statistik: {f_stat:.4f}")
                print(f"P-Wert: {p_val:.4f}")
                
                # Effektstärken
                categories = sorted(analysis_data[pred].unique())
                print("\nEffektstärken (Cohen's d):")
                for i in range(len(categories)):
                    for j in range(i + 1, len(categories)):
                        cat1 = categories[i]
                        cat2 = categories[j]
                        group1 = analysis_data[analysis_data[pred] == cat1][target_variable]
                        group2 = analysis_data[analysis_data[pred] == cat2][target_variable]
                        d = (group1.mean() - group2.mean()) / np.sqrt(
                            ((group1.std()**2 + group2.std()**2) / 2))
                        print(f"Kategorie {cat1} vs {cat2}: {d:.4f}")
        else:
            correlation = analysis_data[target_variable].corr(analysis_data[pred])
            print(f"\nKorrelation mit Zielvariable: {correlation:.4f}")
            
            # ANOVA (für metrische Prädiktoren)
            groups = [group for _, group in analysis_data.groupby(pred)[target_variable]]
            if len(groups) >= 2:
                f_stat, p_val = stats.f_oneway(*groups)
                print(f"\nANOVA Test:")
                print(f"F-Statistik: {f_stat:.4f}")
                print(f"P-Wert: {p_val:.4f}")
    
    # Visualisierungen
    fig, axes = plt.subplots(len(predictor_variables), 2, figsize=(12, 6*len(predictor_variables)))
    if len(predictor_variables) == 1:
        axes = axes.reshape(1, 2)
    
    for i, pred in enumerate(predictor_variables):
        is_categorical = analysis_data[pred].nunique() < 10
        
        if is_categorical:
            # Boxplot
            sns.boxplot(x=pred, y=target_variable, data=analysis_data, ax=axes[i,0])
            axes[i,0].set_title(f'Boxplot: {target_variable} nach {pred}')
            axes[i,0].tick_params(axis='x', rotation=45)
            
            # Violinplot
            sns.violinplot(x=pred, y=target_variable, data=analysis_data, ax=axes[i,1])
            axes[i,1].set_title(f'Verteilung: {target_variable} nach {pred}')
            axes[i,1].tick_params(axis='x', rotation=45)
        else:
            # Streudiagramm
            sns.scatterplot(x=pred, y=target_variable, data=analysis_data, ax=axes[i,0])
            axes[i,0].set_title(f'Streudiagramm: {target_variable} vs {pred}')
            
            # Regression
            sns.regplot(x=pred, y=target_variable, data=analysis_data, ax=axes[i,1])
            axes[i,1].set_title(f'Regression: {target_variable} vs {pred}')
    
    plt.tight_layout()
    plt.show()
    
    return analysis_data

Der dann durch drei Aufrufe nach den drei Zielvariablen Aufrufe, Klickrate und durchschn. Wiedergabedauer abgearbeitet wurde.

In [ ]:
# Beispielaufrufe:
# Für Klickrate mit IQR-Methode
flexible_analysis(X_train, y_train,
                  target_variable='Klickrate der Impressionen (%)',
                  predictor_variables=['Gestaltung_Thumbnail', 'Bewertung_Titel', 'video_length_seconds'],
                  outlier_method='iqr',
                  outlier_threshold=1.5)

In [ ]:
# Für Wiedergabedauer mit modifiziertem Z-Score
flexible_analysis(X_train, y_train,
                  target_variable='durchschnittliche_wiedergabedauer',
                  predictor_variables=['Gestaltung_Thumbnail', 'Bewertung_Titel', 'video_length_seconds'],
                  outlier_method='modified_zscore',
                  #outlier_method='iqr',                  
                  outlier_threshold=3.5)

In [ ]:
# Analyse für Views
# Für Aufrufe mit Z-Score
flexible_analysis(X_train, y_train,
                  target_variable='aufrufe',
                  predictor_variables=['Gestaltung_Thumbnail', 'Bewertung_Titel', 'video_length_seconds'],
                  outlier_method='zscore',
                  outlier_threshold=3)

### Ergebnis EDA:
Die Analyse der drei zentralen Metriken - Wiedergabedauer, Klickrate und Aufrufe - zeigt deutlich unterschiedliche Muster und Einflussfaktoren. Bei allen drei Analysen wurde die Ausreisser mit verschiedenen Methoden (IQR, ZScaler)

**Durchschnittliche Wiedergabedauer:**
Die durchschnittliche Wiedergabedauer beträgt 99,3 Sekunden mit einer beträchtlichen Streuung (Standardabweichung: 48,4 Sekunden). Die Thumbnail-Gestaltung zeigt einen hochsignifikanten Einfluss (p < 0.001), wobei besonders der Unterschied zwischen Kategorie 1 (75,2 Sek.) und den Kategorien 2 und 3 (108,4 bzw. 108,9 Sek.) auffällt. Die Effektstärken (Cohen's d) bestätigen diesen starken Effekt mit Werten von -0,74 und -0,80 zwischen Kategorie 1 und den anderen Kategorien. Bemerkenswert ist auch die starke positive Korrelation mit der Videolänge (r = 0,812), was einen klaren linearen Zusammenhang aufzeigt. Die Titelbewertung zeigt einen signifikanten, aber schwächeren Einfluss (p = 0,010), wobei interessanterweise niedrigere Bewertungen mit längeren Wiedergabezeiten assoziiert sind. Die Violin-Plots zeigen zudem eine zunehmende Konzentration der Verteilung bei höheren Thumbnail-Kategorien, was auf konsistentere Ergebnisse hindeutet.


**Durchschnittliche Wiedergabedauer (Sekunden)**
| Statistische Kennzahl | Wert |
|----------------------|------|
| Mittelwert | 99.30 |
| Standardabweichung | 48.38 |
| Minimum | 24.00 |
| Maximum | 254.00 |
| Median | 87.00 |
| 25%-Quantil | 62.00 |
| 75%-Quantil | 128.00 |

*Nach Thumbnail-Kategorie:*
| Kategorie | Mittelwert | Std.Abw. | n |
|-----------|------------|----------|-----|
| 1 | 75.22 | 39.11 | 227 |
| 2 | 108.41 | 50.43 | 375 |
| 3 | 108.88 | 44.84 | 214 |

ANOVA: F = 42.98, p < 0.001

*Nach Bewertung_Titel:*
| Bewertung | Mittelwert | Std.Abw. | n |
|-----------|------------|----------|-----|
| 4.5 | 149.67 | 88.84 | 3 |
| 5.5 | 104.00 | 55.35 | 6 |
| 6.5 | 102.56 | 47.63 | 574 |
| 7.5 | 93.26 | 50.59 | 129 |
| 8.0 | 78.55 | 21.76 | 11 |
| 8.5 | 88.10 | 47.73 | 93 |

ANOVA: F = 3.02, p = 0.010

*Korrelationen:*
- Mit Video-Länge: r = 0.812
- Mit Bewertung_Titel: r = -0.127 (berechnet aus den Gruppenmittelwerten)
- Mit Gestaltung_Thumbnail: r = 0.301 (berechnet aus den Gruppenmittelwerten)

Wichtige Erkenntnisse:
1. Die Ausreißerbehandlung hat zu deutlich robusteren und klareren Ergebnissen geführt
2. Der Einfluss der Thumbnail-Gestaltung ist noch deutlicher geworden:
   - Klare Überlegenheit der Kategorien 2 und 3
   - Praktisch kein Unterschied zwischen Kategorie 2 und 3
3. Überraschender inverser Zusammenhang zwischen Titelbewertung und Wiedergabedauer


<img src="../references/images/Grafische auswertung zielvariable Wiedergabedauer.JPG" width="800" alt="Grafische Analyse Klickrate"/>


**Klickrate der Impressionen:**
Die durchschnittliche Klickrate liegt bei 3,98% mit einer typischen Streuung (Standardabweichung: 2,14%). Die Verteilung zeigt eine realistische Spanne von 1% bis 10,05%, wobei der Median bei 3,52% liegt. Die Thumbnail-Gestaltung erweist sich als signifikanter Einflussfaktor (p = 0,038), wobei besonders die Kategorie 3 mit einem Durchschnitt von 4,24% positiv hervorsticht. Interessanterweise zeigt die Titelbewertung keinen statistisch signifikanten Einfluss (p = 0,737), obwohl die mittleren Bewertungskategorien (6,5 und 7,5) tendenziell etwas höhere Klickraten aufweisen. Die Videolänge hat einen sehr schwachen, leicht negativen Zusammenhang mit der Klickrate (r = -0,024), was darauf hindeutet, dass kürzere Videos minimal bessere Klickraten erzielen, dieser Effekt aber praktisch kaum relevant ist. Besonders auffällig ist die Konzentration der Daten im mittleren Bereich, was sich in der relativ geringen Differenz zwischen dem 25%- und 75%-Quantil (2,29% bis 5,34%) widerspiegelt.


**Klickrate der Impressionen (%)**
| Statistische Kennzahl | Wert |
|----------------------|------|
| Mittelwert | 3.98 |
| Standardabweichung | 2.14 |
| Minimum | 1.00 |
| Maximum | 10.05 |
| Median | 3.52 |
| 25%-Quantil | 2.29 |
| 75%-Quantil | 5.34 |

*Nach Thumbnail-Kategorie:*
| Kategorie | Mittelwert | Std.Abw. | n |
|-----------|------------|----------|-----|
| 1 | 4.04 | 2.14 | 228 |
| 2 | 3.80 | 2.13 | 393 |
| 3 | 4.24 | 2.13 | 226 |

ANOVA: F = 3.27, p = 0.038

*Nach Bewertung_Titel:*
| Bewertung | Mittelwert | Std.Abw. | n |
|-----------|------------|----------|-----|
| 4.5 | 3.81 | 2.31 | 2 |
| 5.5 | 3.29 | 1.79 | 8 |
| 6.5 | 4.02 | 2.14 | 593 |
| 7.5 | 4.05 | 2.18 | 133 |
| 8.0 | 3.44 | 1.91 | 14 |
| 8.5 | 3.80 | 2.12 | 97 |

ANOVA: F = 0.55, p = 0.737

*Korrelationen:*
- Mit Video-Länge: r = -0.024
- Mit Bewertung_Titel: r = -0.027 (berechnet aus den Gruppenmittelwerten)
- Mit Gestaltung_Thumbnail: r = 0.048 (berechnet aus den Gruppenmittelwerten)

Die Plots sahen ohne Ausreisser viel Spitzer aus. 
Schlussfolgerungen:
1. Die Ausreißerbehandlung hat die Analyse geschärft und zu robusteren statistischen Ergebnissen geführt
2. Der Einfluss der Thumbnail-Gestaltung ist nun statistisch signifikanter nachweisbar

<img src="../references/images/Grafische auswertung zielvariable Klickrate.JPG" width="800" alt="Grafische Analyse durchschnittliche Wiedergabedauer"/>


**Aufrufe:**
Die Aufrufe zeigen eine sehr breite Streuung mit einem Mittelwert von 36.346 und einer hohen Standardabweichung von 53.476, was auf eine stark rechtsschiefe Verteilung hinweist. Der Median von 13.615 Aufrufen liegt deutlich unter dem Mittelwert, was diese Schiefe bestätigt. Die Thumbnail-Gestaltung zeigt einen knapp nicht signifikanten Einfluss (p = 0.056), wobei Kategorie 3 mit durchschnittlich 43.642 Aufrufen merklich höhere Werte erzielt als die Kategorien 1 und 2 (ca. 33.000). Bemerkenswert ist die sehr schwache Korrelation mit der Videolänge (r = 0.021), was darauf hindeutet, dass die Länge eines Videos praktisch keinen Einfluss auf die Aufrufzahlen hat. Die Titelbewertung zeigt keinen signifikanten Einfluss (p = 0.498), wobei die geringen Fallzahlen in einigen Bewertungskategorien die Aussagekraft einschränken. Die Violin-Plots verdeutlichen die extreme Streuung der Aufrufe über alle Kategorien hinweg, was auf den starken Einfluss externer Faktoren hindeutet.


**Aufrufe**
| Statistische Kennzahl | Wert |
|----------------------|------|
| Mittelwert | 36.346 |
| Standardabweichung | 53.476 |
| Minimum | 178 |
| Maximum | 293.153 |
| Median | 13.615 |
| 25%-Quantil | 3.227 |
| 75%-Quantil | 46.170 |

*Nach Thumbnail-Kategorie:*
| Kategorie | Mittelwert | Std.Abw. | n |
|-----------|------------|----------|-----|
| 1 | 33.176 | 50.445 | 229 |
| 2 | 33.993 | 52.591 | 389 |
| 3 | 43.642 | 57.401 | 225 |

ANOVA: F = 2.89, p = 0.056

*Nach Bewertung_Titel:*
| Bewertung | Mittelwert | Std.Abw. | n |
|-----------|------------|----------|-----|
| 4.5 | 71.906 | 66.096 | 3 |
| 5.5 | 42.450 | 87.460 | 8 |
| 6.5 | 36.084 | 52.790 | 590 |
| 7.5 | 41.410 | 58.746 | 133 |
| 8.0 | 23.106 | 40.972 | 14 |
| 8.5 | 31.198 | 47.741 | 95 |

ANOVA: F = 0.87, p = 0.498

*Korrelationen:*
- Mit Video-Länge: r = 0.021
- Mit Bewertung_Titel: r = -0.183 (berechnet aus den Gruppenmittelwerten)
- Mit Gestaltung_Thumbnail: r = 0.097 (berechnet aus den Gruppenmittelwerten)

Wichtige Erkenntnisse:
1. Die Ausreißerbehandlung hat erstmals signifikante Effekte der Thumbnail-Gestaltung auf die Aufrufe sichtbar gemacht
2. Die hohe Variabilität der Aufrufe bleibt bestehen, was auf starke externe Einflussfaktoren hindeutet
3. Die Titelbewertung zeigt keinen klaren Zusammenhang mit den Aufrufzahlen

<img src="../references/images/Grafische auswertung zielvariable aufrufe.JPG" width="800" alt="Grafische Analyse Aufrufe"/>


Ein besonders interessanter Aspekt ist die unterschiedliche Wirkung der Thumbnail-Gestaltung auf die verschiedenen Metriken: Während sie bei der Wiedergabedauer einen starken und bei der Klickrate einen moderaten Einfluss hat, scheint sie für die absolute Anzahl der Aufrufe kaum relevant zu sein. Dies könnte darauf hindeuten, dass gut gestaltete Thumbnails zwar das Engagement der Zuschauer fördern, die reine Reichweite aber von anderen Faktoren abhängt.

Diese Erkenntnisse legen nahe, dass für die verschiedenen Erfolgskennzahlen unterschiedliche Optimierungsstrategien erforderlich sind und eine reine Fokussierung auf einzelne Gestaltungsaspekte nicht ausreicht, um den Gesamterfolg eines Videos zu gewährleisten.

## Methodology

#### Modellierungsansatz für  Regressionsanalyse
Den besten Ansatz nach der EDA sehe ich in der durchschnittlichen Wiedergabedauer. Dazu habe ich auch noch die Variable Publishing_Date in Wochentag und Stunde aufgeschlüsselt. Hier könnte auch noch ein sinnvoller Prädiktor liegen. Im Code wird alles ine einem Durchlauf analysiert. Die Ausreisser werden eleminiert, Stunde und Wochentag extrahiert und auch eine Cross-Validation zwischen Trainings- und Validierungsdatensatz durchgeführt. Die Modelle werde ich später letzendlich mit den Testdatensatz endgültig auf Herz und Nieren prüfen. 


#### Hier der Code für eine Regressionsanalyse der durchscnittlichen Wiedergabedauer

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

def load_and_filter_data():
    """Lädt die Pickle-Daten und filtert die relevanten Features"""
    with open('../data/processed/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)

    # Zeitliche Features aus publish_date extrahieren
    for dataset in ['X_train', 'X_val', 'X_test']:
        if 'publish_date' in data[dataset].columns:
            # Konvertiere zu datetime mit deutschem Format
            data[dataset]['publish_date'] = pd.to_datetime(data[dataset]['publish_date'], 
                                                         format='%d.%m.%Y %H:%M')
            # Extrahiere Stunde und Wochentag
            data[dataset]['publish_hour'] = data[dataset]['publish_date'].dt.hour
            data[dataset]['publish_weekday'] = data[dataset]['publish_date'].dt.dayofweek
    
    # Liste der zu behaltenden Features
    keep_features = [
        'video_length_seconds',
        'Gestaltung_Thumbnail',
        'Bewertung_Titel',
        'publish_hour',
        'publish_weekday'
    ]
    
    # Thema-Dummy-Variablen finden und hinzufügen
    #theme_features = [col for col in data['X_train'].columns if col.startswith('theme_')]
    # Thema-Dummy-Variablen finden und hinzufügen, aber ohne "Live"-Dummy, da ja auch keine Live-Videos mehr dabei sind
    theme_features = [col for col in data['X_train'].columns if col.startswith('theme_') and not col.endswith('Live')]
    keep_features.extend(theme_features)
    
    # Features filtern
    X_train = data['X_train'][keep_features]
    X_val = data['X_val'][keep_features]
    X_test = data['X_test'][keep_features]
    
    # Zielvariable
    y_train = data['y_train']['durchschnittliche_wiedergabedauer']
    y_val = data['y_val']['durchschnittliche_wiedergabedauer']
    y_test = data['y_test']['durchschnittliche_wiedergabedauer']
    
    print("Verwendete Features:")
    print(X_train.columns.tolist())
    
    return X_train, X_val, X_test, y_train, y_val, y_test

def remove_outliers(X, y, method='iqr', threshold=1.5):
    """Entfernt Ausreißer basierend auf der gewählten Methode"""
    if method == 'iqr':
        Q1 = y.quantile(0.25)
        Q3 = y.quantile(0.75)
        IQR = Q3 - Q1
        mask = ~((y < (Q1 - threshold * IQR)) | (y > (Q3 + threshold * IQR)))
    elif method == 'zscore':
        z_scores = np.abs(stats.zscore(y))
        mask = z_scores < threshold
    
    X_clean = X[mask]
    y_clean = y[mask]
    
    print(f"Ausreißerbehandlung mit Methode: {method}, Schwellenwert: {threshold}")
    print(f"Entfernte Datenpunkte: {len(y) - len(y_clean)} ({(1 - len(y_clean)/len(y))*100:.1f}%)")
    
    return X_clean, y_clean

def train_models(X, y):
    """Trainiert verschiedene Regressionsmodelle und vergleicht ihre Performance"""
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Lasso Regression': Lasso(alpha=1.0)
    }
    
    results = {}
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for name, model in models.items():
        # Cross-Validation Scores
        cv_scores = cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_squared_error')
        rmse_scores = np.sqrt(-cv_scores)
        
        # Modell auf gesamten Datensatz trainieren
        model.fit(X, y)
        y_pred = model.predict(X)
        
        results[name] = {
            'model': model,
            'cv_rmse_mean': rmse_scores.mean(),
            'cv_rmse_std': rmse_scores.std(),
            'r2_score': r2_score(y, y_pred),
            'mae': mean_absolute_error(y, y_pred),
            'predictions': y_pred
        }
        
        # Feature Importance für Linear und Ridge Regression
        if name in ['Linear Regression', 'Ridge Regression']:
            results[name]['feature_importance'] = pd.DataFrame({
                'feature': X.columns,
                'importance': model.coef_
            }).sort_values('importance', key=lambda x: abs(x), ascending=False)  # Sortierung nach absolutem Wert
    
    return results

def analyze_residuals(y_true, y_pred, title="Residuenanalyse"):
    """Führt eine detaillierte Residuenanalyse durch"""
    residuals = y_true - y_pred
    
    # Plot erstellen
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(title)
    
    # Residuen vs. vorhergesagte Werte
    axes[0,0].scatter(y_pred, residuals, alpha=0.5)
    axes[0,0].axhline(y=0, color='r', linestyle='--')
    axes[0,0].set_xlabel('Vorhergesagte Werte')
    axes[0,0].set_ylabel('Residuen')
    axes[0,0].set_title('Residuen vs. Vorhergesage')
    
    # Q-Q Plot
    stats.probplot(residuals, dist="norm", plot=axes[0,1])
    axes[0,1].set_title('Q-Q Plot')
    
    # Histogramm der Residuen
    axes[1,0].hist(residuals, bins=30)
    axes[1,0].set_xlabel('Residuen')
    axes[1,0].set_ylabel('Häufigkeit')
    axes[1,0].set_title('Verteilung der Residuen')
    
    # Tatsächliche vs. vorhergesagte Werte
    axes[1,1].scatter(y_true, y_pred, alpha=0.5)
    axes[1,1].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--')
    axes[1,1].set_xlabel('Tatsächliche Werte')
    axes[1,1].set_ylabel('Vorhergesagte Werte')
    axes[1,1].set_title('Vorhersage vs. Realität')
    
    plt.tight_layout()
    plt.show()
    
    # Statistiken der Residuen
    print("\nStatistiken der Residuen:")
    print(f"Mittelwert: {residuals.mean():.4f}")
    print(f"Standardabweichung: {residuals.std():.4f}")
    print(f"Schiefe: {stats.skew(residuals):.4f}")
    print(f"Kurtosis: {stats.kurtosis(residuals):.4f}")
    
    # Shapiro-Wilk Test auf Normalverteilung
    _, p_value = stats.shapiro(residuals)
    print(f"\nShapiro-Wilk Test p-Wert: {p_value:.4f}")
    
    # Breusch-Pagan Test auf Homoskedastizität
    _, p_value = stats.levene(residuals, y_pred)
    print(f"Breusch-Pagan Test p-Wert: {p_value:.4f}")

def main():
    # 1) Daten laden und filtern
    X_train, X_val, X_test, y_train, y_val, y_test = load_and_filter_data()
    
    # 2) Ausreißer entfernen
    X_train_clean, y_train_clean = remove_outliers(X_train, y_train, method='iqr', threshold=1.5)
    
    # 3) Modelle trainieren
    results = train_models(X_train_clean, y_train_clean)
    
    # 4) Ergebnisse ausgeben
    print("\nModellvergleich:")
    for name, result in results.items():
        print(f"\n{name}:")
        print(f"Cross-validated RMSE: {result['cv_rmse_mean']:.2f} (+/- {result['cv_rmse_std']:.2f})")
        print(f"R² Score: {result['r2_score']:.4f}")
        print(f"MAE: {result['mae']:.4f}")
        
        if 'feature_importance' in result:
            print("\nFeature Importance:")
            print(result['feature_importance'])
    
    # 5) Bestes Modell auswählen
    best_model_tuple = max(results.items(), key=lambda x: x[1]['r2_score'])  # CHANGED: rename to _tuple
    best_model_name = best_model_tuple[0]   # z.B. "Linear Regression"
    best_model_info = best_model_tuple[1]   # dictionary mit {'model':..., 'predictions':..., ...}

    sklearn_model = best_model_info['model']  # ADDED: Hol dir das Modellobjekt
    print(f"\nBestes Modell: {best_model_name}")
    
    # 6) Validierung
    final_predictions = sklearn_model.predict(X_val)
    val_r2 = r2_score(y_val, final_predictions)
    val_rmse = np.sqrt(mean_squared_error(y_val, final_predictions))
    print(f"\nValidierungsergebnisse:")
    print(f"R² Score: {val_r2:.4f}")
    print(f"RMSE: {val_rmse:.2f}")
    
    # 7) Residuenanalyse
    analyze_residuals(y_train_clean, best_model_info['predictions'], 
                     title=f"Residuenanalyse - {best_model_name}")
    
    # 8) Intercept & Coefficients für lineare Modelle ausgeben - ADDED
    if hasattr(sklearn_model, 'coef_') and hasattr(sklearn_model, 'intercept_'):
        intercept = sklearn_model.intercept_
        coefficients = sklearn_model.coef_
        features = X_train_clean.columns
        
        print("\n--- Lineare Modellformel / Koeffizienten ---")
        print(f"Intercept: {intercept:.4f}")
        
        for feat, coef_val in zip(features, coefficients):
            print(f"{feat}: {coef_val:.4f}")
    
    # 9) Rückgabe (unverändert, nur best_model_tuple -> best_model_info['model'])
    return results, sklearn_model  # CHANGED: Return the sklearn_model directly

    

if __name__ == "__main__":
    results, best_model = main()

#### Die Ergebnisse der ersten Regressionsanalyse (durchschnittliche Wiedergabedauer):

**Modellvergleich**

| Modell | RMSE (CV) | R² | MAE | Validation R² | Validation RMSE |
|--------|-----------|-----|-----|---------------|-----------------|
| Linear Regression | 27.18 (±2.96) | 0.6736 | 19.5105 | 0.7776 | 37.34 |
| Ridge Regression | 27.18 (±2.96) | 0.6736 | 19.5095 | - | - |
| Lasso Regression | 27.35 (±3.09) | 0.6672 | 19.6472 | - | - |

**Residualstatistiken**

| Metrik | Wert |
|--------|------|
| Mittelwert | -0.0000 |
| Standardabweichung | 26.7346 |
| Schiefe | -0.1785 |
| Kurtosis | 3.3163 |

**Modellkoeffizienten**

| Parameter | Koeffizient |
|-----------|-------------|
| Intercept | 33.2770 |
| video_length_seconds | 0.2053 |
| Gestaltung_Thumbnail | 3.7091 |
| Bewertung_Titel | 0.5780 |
| publish_hour | 0.0340 |
| publish_weekday | -0.0379 |
| theme_Bilder | -12.0715 |
| theme_Krieg | 10.7072 |
| theme_Politik | 7.1267 |
| theme_Sonstiges | -6.3158 |
| theme_Wirtschaft | 0.5534 |

**Feature Importance (sortiert nach absolutem Einfluss)**

| Feature | Importance |
|---------|------------|
| theme_Bilder | -12.071480 |
| theme_Krieg | 10.707203 |
| theme_Politik | 7.126674 |
| theme_Sonstiges | -6.315753 |
| Gestaltung_Thumbnail | 3.709104 |
| Bewertung_Titel | 0.577990 |
| theme_Wirtschaft | 0.553356 |
| video_length_seconds | 0.205289 |
| publish_weekday | -0.037912 |
| publish_hour | 0.033981 |


<img src="../references/images/modell1wiedergabedauer.JPG" width="800" alt="Modell 1 druchschn. Wiedergabedauer"/>


**Zusammenfassung der wichtigsten Erkenntnisse**

| Aspekt | Erkenntnis |
|--------|------------|
| Modellgüte | Moderate bis gute Erklärungskraft (R² = 0.67) |
| Stärkste positive Prädiktoren | Kriegsthemen (+10.71s), Politik (+7.13s) |
| Stärkste negative Prädiktoren | Bilderthemen (-12.07s), Sonstige Themen (-6.32s) |
| Thumbnail-Effekt | Positiver Einfluss (+3.71s) |
| Zeitliche Faktoren | Vernachlässigbarer Einfluss |
| Modellannahmen | Verletzung der Normalverteilung und Homoskedastizität |

**Analyse der linearen Regressionsmodelle**

Die Regressionsanalyse für die Wiedergabedauer von YouTube-Videos zeigt interessante und praxisrelevante Ergebnisse. Das lineare Regressionsmodell erklärt etwa 67% der Varianz in den Trainingsdaten (R² = 0.6736) und zeigt sogar eine bessere Performance im Validierungsdatensatz (R² = 0.7776). Dies deutet auf ein robustes Modell hin, das nicht überangepasst ist.

Besonders aufschlussreich sind die identifizierten Einflussfaktoren:
- Thematische Ausrichtung hat den stärksten Einfluss: Kriegs- und Politikthemen führen zu deutlich längeren Wiedergabezeiten (+10.71s bzw. +7.13s), während Bilderthemen die Wiedergabedauer stark reduzieren (-12.07s)
- Die Thumbnail-Gestaltung zeigt einen beachtlichen positiven Effekt (+3.71s)
- Überraschend ist der geringe Einfluss zeitlicher Faktoren wie Veröffentlichungsstunde oder Wochentag

Der Vergleich verschiedener Modellvarianten (Linear, Ridge, Lasso) zeigt sehr ähnliche Performancewerte, was die Stabilität der Ergebnisse unterstreicht. Die durchschnittliche Vorhersageabweichung von etwa 27 Sekunden (RMSE) ist im Kontext von YouTube-Videos als akzeptabel zu bewerten.

Für die praktische Content-Strategie legen diese Ergebnisse nahe, dass der thematische Fokus und die Thumbnail-Gestaltung die wichtigsten Stellhebel für die Optimierung der Wiedergabedauer sind, während der Veröffentlichungszeitpunkt eine untergeordnete Rolle spielt. Die identifizierten Verletzungen der Modellannahmen sollten bei der Interpretation berücksichtigt werden, beeinträchtigen aber nicht die grundsätzlichen strategischen Implikationen.

## Results

> REMOVE THE FOLLOWING TEXT

This is where you will output the final model with any relevant model fit statistics.

Describe the key results from the model.
The goal is not to interpret every single variable in the model but rather to show that you are proficient in using the model output to address the research questions, using the interpretations to support your conclusions.

Focus on the variables that help you answer the research question and that provide relevant context for the reader.


#### **Forward Feature Selection**, (für ausgewählte Prädiktoren)

Die Analyse konzentriert sich auf Faktoren, die bei der Erstellung von YouTube-Videos aktiv beeinflusst werden können. Dazu gehören:

- Die Länge des Videos
- Das gewählte Thema 
- Die Gestaltung des Titels
- Die Gestaltung des Thumbnails
- Der Veröffentlichungszeitpunkt (Wochentag und Uhrzeit)

Um die Bedeutung dieser Faktoren systematisch zu untersuchen, wurde ein schrittweises Verfahren gewählt. Dabei startet die Analyse mit einem leeren Modell. In jedem Schritt wird dann jene Variable hinzugefügt, die die größte Verbesserung der Vorhersagegenauigkeit bringt. Diese Verbesserung wird durch Kreuzvalidierung überprüft, um zufällige Effekte auszuschließen.

Dieser Ansatz hat den Vorteil, dass er die Faktoren nach ihrer Wichtigkeit ordnet. So lässt sich erkennen, welche Gestaltungselemente den stärksten Einfluss auf den Erfolg eines Videos haben. Andere Variablen, wie etwa bereits vorhandene Klickzahlen oder automatisch erfasste Metriken, wurden bewusst nicht berücksichtigt, da sie für die praktische Videogestaltung nicht relevant sind.

Die Ergebnisse dieser Analyse ermöglichen es, Ressourcen gezielt dort einzusetzen, wo sie den größten Effekt haben. Wenn sich zum Beispiel zeigt, dass die Thumbnail-Gestaltung einen besonders starken Einfluss hat, während der Veröffentlichungszeitpunkt kaum eine Rolle spielt, können entsprechende Prioritäten in der Content-Produktion gesetzt werden.



##### Code für Forward Selection

In [ ]:
import os
import json
import joblib
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from itertools import combinations

#TARGET_VAR = "durchschnittliche_wiedergabedauer"  # Zielvariable ist durchschn. Wiedergabedauer
#TARGET_VAR = "aufrufe"  # Aufrufe (views)
TARGET_VAR = "Klickrate der Impressionen (%)"  # Klickrate der Impressionen

def forward_feature_selection(X, y, 
                              scoring='r2', 
                              n_splits=5, 
                              random_state=42):
    """
    Führt eine Vorwärtsselektion (Forward Feature Selection) durch,
    basierend auf dem ausgewählten Scoring (Standard: R²).
    
    :param X: pd.DataFrame mit den rein beeinflussbaren Features
    :param y: pd.Series oder np.array mit der Zielvariable
    :param scoring: 'r2' oder 'neg_mean_squared_error'
    :param n_splits: Anzahl K-Folds in der Cross-Validation
    :param random_state: Zufallssamen für Reproduzierbarkeit
    :return: 
       selected_features (List): Features in der Reihenfolge ihrer Aufnahme
       best_models (Dict): Zwischenergebnisse 
           Key = Tuple(Features), Value = (Train_R2, Train_RMSE, Train_MAE, Modellobjekt)
    """
    
    all_features = list(X.columns)  # Alle möglichen Features, die DU beeinflussen kannst
    
    selected_features = []          # Start: Keine Features
    remaining_features = set(all_features)
    best_models = {}
    
    while remaining_features:
        best_score = -np.inf
        best_feature = None
        
        # Teste jedes Feature, das noch nicht ausgewählt ist
        for feature in remaining_features:
            current_features = selected_features + [feature]
            X_sub = X[current_features]
            
            model = LinearRegression()
            
            # Cross-Validation (für R² oder neg_mean_squared_error)
            kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
            
            if scoring == 'r2':
                cv_scores = cross_val_score(model, X_sub, y, cv=kfold, scoring='r2')
                mean_score = cv_scores.mean()
            elif scoring == 'neg_mean_squared_error':
                cv_scores = cross_val_score(model, X_sub, y, cv=kfold, scoring='neg_mean_squared_error')
                # Hier sind die Werte negativ, da MSE "minimiert" wird. 
                # Man könnte -mean_score nehmen, um "maximieren" zu simulieren. 
                # Für Forward-Selection reicht es oft, den Mittelwert direkt zu vergleichen.
                mean_score = cv_scores.mean()
            else:
                raise ValueError("Bitte scoring='r2' oder 'neg_mean_squared_error' verwenden.")
            
            # Wähle jenes Feature, das den besten (höchsten) Score liefert
            if mean_score > best_score:
                best_score = mean_score
                best_feature = feature
        
        if best_feature is not None:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
            
            # Trainiere ein finales Modell auf dem gesamten Datensatz (Train) mit den ausgewählten Features
            X_sub = X[selected_features]
            final_model = LinearRegression()
            final_model.fit(X_sub, y)
            
            # Metriken auf dem gesamten Trainingsset (optional, zur Info)
            y_pred = final_model.predict(X_sub)
            train_r2 = r2_score(y, y_pred)
            train_rmse = np.sqrt(mean_squared_error(y, y_pred))
            train_mae = np.mean(np.abs(y - y_pred))
            
            best_models[tuple(selected_features)] = (train_r2, train_rmse, train_mae, final_model)
            
            print(f"Feature hinzugefügt: {best_feature}. "
                  f"Aktuelle Liste: {selected_features}. "
                  f"(Train R²={train_r2:.4f}, RMSE={train_rmse:.2f}, MAE={train_mae:.2f})")
        else:
            # Wenn kein Feature den Score steigert, brechen wir
            break
    
    return selected_features, best_models

def main():
    # 1) Daten laden
    with open('../data/processed/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
    
    # Zeitliche Features aus publish_date extrahieren
    for dataset in ['X_train', 'X_val']:
        if 'publish_date' in data[dataset].columns:
            data[dataset]['publish_date'] = pd.to_datetime(
                data[dataset]['publish_date'], 
                format='%d.%m.%Y %H:%M'
            )
            data[dataset]['publish_hour'] = data[dataset]['publish_date'].dt.hour
            data[dataset]['publish_weekday'] = data[dataset]['publish_date'].dt.dayofweek
    
    # 2) Trainings- und Validierungsdaten laden
    X_train = data['X_train'].copy()
    y_train = data['y_train'][TARGET_VAR].copy()
    
    X_val = data['X_val'].copy()
    y_val = data['y_val'][TARGET_VAR].copy()
    
    # 3) Nur beeinflussbare Features (ohne theme_Live) behalten
    candidate_features = [
        'video_length_seconds',
        'Gestaltung_Thumbnail',
        'Bewertung_Titel',
        'publish_hour',
        'publish_weekday',
        'theme_Bilder',
        'theme_Krieg',
        'theme_Politik',
        'theme_Sonstiges',
        'theme_Wirtschaft'
    ]
    
    # Falls "theme_Live" existiert, ausschließen
    if 'theme_Live' in X_train.columns:
        X_train.drop(columns=['theme_Live'], inplace=True, errors='ignore')
    if 'theme_Live' in X_val.columns:
        X_val.drop(columns=['theme_Live'], inplace=True, errors='ignore')
    
    # Filter auf die Candidate-Features
    X_train = X_train[candidate_features]
    X_val = X_val[candidate_features]
    
    # 4) Forward Feature Selection (auf Trainingsdaten)
    selected_feats, models_dict = forward_feature_selection(
        X_train, 
        y_train, 
        scoring='r2',           # oder 'neg_mean_squared_error'
        n_splits=5, 
        random_state=42
    )
    
    print("\n--- Vorwärtsselektion abgeschlossen ---")
    print("Ausgewählte Features (in Reihenfolge der Aufnahme):")
    for feat in selected_feats:
        print("  ", feat)
    
    # 5) Bestes Modell anhand des höchsten Train-R²
    best_key = max(models_dict.keys(), key=lambda k: models_dict[k][0])  # Sortiere nach R²
    best_r2, best_rmse, best_mae, best_model = models_dict[best_key]
    
    # 6) Test des besten Modells auf dem Validierungsdatensatz
    X_val_sub = X_val[list(best_key)]
    y_val_pred = best_model.predict(X_val_sub)
    
    val_r2 = r2_score(y_val, y_val_pred)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    val_mae = mean_absolute_error(y_val, y_val_pred)
    
    print("\n--- Validierungsergebnisse mit dem besten Modell ---")
    print(f"Train R² = {best_r2:.4f}, Val R² = {val_r2:.4f}")
    print(f"Train RMSE = {best_rmse:.2f}, Val RMSE = {val_rmse:.2f}")
    print(f"Train MAE = {best_mae:.2f}, Val MAE = {val_mae:.2f}")
    
    # 7) Ausgabe der Koeffizienten (optional)
    coef_df = pd.DataFrame({
        'Feature': list(best_key),
        'Coefficient': best_model.coef_
    })
    print("\nKoeffizienten des finalen Modells:")
    print("Intercept:", best_model.intercept_)
    print(coef_df)
    
    # -----------------------------------
    # 8) MODEL & INFO SPEICHERN
    # -----------------------------------
    # Ordner "models" anlegen, falls nicht vorhanden
    os.makedirs("../models", exist_ok=True)
    
    # Model-Dateiname (z.B. best_model_durchschnittliche_wiedergabedauer.pkl)
    model_filename = f"../models/best_model_{TARGET_VAR}.pkl"
    joblib.dump(best_model, model_filename)
    print(f"\nModell gespeichert in: {model_filename}")
    
    # Wichtigste Kennzahlen & Parameter als JSON speichern
    model_info = {
        "target_variable": TARGET_VAR,
        "selected_features": list(best_key),
        "train_r2": round(best_r2, 4),
        "train_rmse": round(best_rmse, 2),
        "train_mae": round(best_mae, 2),
        "val_r2": round(val_r2, 4),
        "val_rmse": round(val_rmse, 2),
        "val_mae": round(val_mae, 2),
        "coefficients": {
            "Intercept": round(best_model.intercept_, 4)
        }
    }
    # Coefficients-Detail
    for feat, coef_val in zip(best_key, best_model.coef_):
        model_info["coefficients"][feat] = round(float(coef_val), 4)
    
    # JSON-Dateiname (z.B. best_model_durchschnittliche_wiedergabedauer_info.json)
    json_filename = f"../models/best_model_{TARGET_VAR}_info.json"
    with open(json_filename, "w", encoding="utf-8") as f:
        json.dump(model_info, f, indent=2, ensure_ascii=False)
    
    print(f"\nModell-Informationen als JSON gespeichert in: {json_filename}")


if __name__ == "__main__":
    main()

###  Modelle im Vergleich (Model 1 vs. Forward Selection)

| Metrik | Ursprüngliches Modell | Modell nach Vorwärtsselektion |
|--------|----------------------|------------------------------|
| Train R² | 0.6736 | 0.8642 |
| Val R² | 0.7776 | 0.6946 |
| Train RMSE | 27.18 | 35.03 |
| Val RMSE | 37.34 | 43.76 |
| Train MAE | 19.51 | 22.38 |
| Val MAE | - | 22.27 |

**2. Wesentliche Unterschiede**

***Modellperformance***
- Das neue Modell zeigt eine deutlich bessere Performance auf den Trainingsdaten (R² = 0.8642 vs. 0.6736)
- Allerdings ist die Validierungsperformance etwas schlechter (R² = 0.6946 vs. 0.7776)
- Die Fehlerwerte (RMSE, MAE) sind beim neuen Modell leicht höher

**Koeffizientenvergleich**

| Feature | Ursprüngliches Modell | Neues Modell | Änderung |
|---------|---------------------|--------------|----------|
| Intercept | 33.277 | 16.814 | ↓ |
| video_length_seconds | 0.205 | 0.243 | ↑ |
| theme_Bilder | -12.072 | -13.146 | ↓ |
| theme_Krieg | 10.707 | 7.968 | ↓ |
| Gestaltung_Thumbnail | 3.709 | 4.944 | ↑ |
| Bewertung_Titel | 0.578 | 2.269 | ↑↑ |
| publish_hour | 0.034 | -0.063 | ↔ |
| publish_weekday | -0.038 | 0.415 | ↑ |
| theme_Wirtschaft | 0.553 | -2.529 | ↓↓ |

**Praktische Implikationen**

 a) Bestätigte Erkenntnisse
- Thematische Ausrichtung bleibt der wichtigste Einflussfaktor
- Bilderthemen haben weiterhin den stärksten negativen Einfluss
- Gestaltungselemente (Thumbnail, Titel) zeigen positive Effekte

 b) Neue Erkenntnisse
- Stärkerer Einfluss der Titelgestaltung (Koeffizient vervierfacht)
- Deutlichere Auswirkung des Wochentags
- Wirtschaftsthemen zeigen nun einen negativen statt positiven Effekt

**Empfehlungen für die Praxis**

1. **Content-Strategie**
   - Fokus auf Kriegs- und Politikthemen beibehalten (Risiko wenn Krieg vorbei, Politik wenig los)
   - Bilderthemen strategisch einsetzen, aber nicht überstrapazieren
   - Wirtschaftsthemen kritisch prüfen (Themen näher am User suchen)

2. **Produktionsoptimierung**
   - Verstärkter Fokus auf Titeloptimierung
   - Weiterhin hohe Priorität für Thumbnail-Gestaltung
   - Publikationszeitpunkt berücksichtigen, aber nicht überpriorisieren

3. **Monitoring**
   - Regelmäßige Überprüfung der Themenperformance
   - A/B-Tests für Titel- und Thumbnail-Varianten
   - Kontinuierliche Validierung der Modellvorhersagen

Die Regressionsanalyse liefert trotz ihrer Limitationen wertvolle und praktisch anwendbare Erkenntnisse für die Content-Optimierung. Die Kombination beider Modelle erhöht die Verlässlichkeit der grundlegenden Schlussfolgerungen und identifiziert klare Handlungsprioritäten für die Content-Strategie.

## Discussion + Conclusion


### Letzter Vergleich der Modelle für alle Zielvariablen mit Train, Val- und Testdaten
Dafür wurden die einzelnen Forward Slection Analyse im Ordner Models gespeichert. So können diese nun noch einmal mit den drei Sets verglichen werden. 

In [ ]:
import os
import json
import pickle
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def main():
    # 1) Ziele definieren: Du hast 3 Zielvariablen
    target_vars = [
        "Klickrate der Impressionen (%)",
        "durchschnittliche_wiedergabedauer",
        "aufrufe"
    ]
    
    # 2) Daten laden (inkl. X_test, y_test)
    with open('../data/processed/processed_data.pkl', 'rb') as f:
        data = pickle.load(f)
    
    X_test = data['X_test'].copy()
    y_test_data = data['y_test']  # DataFrame oder Series mit allen Zielspalten
    
    # Falls 'publish_date' oder 'theme_Live' etc. noch aufbereitet werden muss, hier tun:
    if 'publish_date' in X_test.columns:
        X_test['publish_date'] = pd.to_datetime(X_test['publish_date'], format='%d.%m.%Y %H:%M')
        X_test['publish_hour'] = X_test['publish_date'].dt.hour
        X_test['publish_weekday'] = X_test['publish_date'].dt.dayofweek
    
    if 'theme_Live' in X_test.columns:
        X_test.drop(columns=['theme_Live'], inplace=True, errors='ignore')
    
    # 3) Ergebnisse sammeln
    results_list = []  # Speichert alle Ergebnisse in einer Liste von Dicts
    
    for target in target_vars:
        # a) JSON-Datei mit den Kennzahlen laden
        json_path = f"../models/best_model_{target}_info.json"
        if not os.path.exists(json_path):
            print(f"Warnung: {json_path} nicht gefunden. Überspringe {target}.")
            continue
        
        with open(json_path, "r", encoding="utf-8") as jf:
            model_info = json.load(jf)
        
        # b) Modell laden (pkl-Datei)
        model_path = f"../models/best_model_{target}.pkl"
        if not os.path.exists(model_path):
            print(f"Warnung: {model_path} nicht gefunden. Überspringe {target}.")
            continue
        
        best_model = joblib.load(model_path)
        
        # c) Benötigte Features laut JSON
        selected_features = model_info.get("selected_features", [])
        
        # d) Test-Zielwerte extrahieren
        if target not in y_test_data.columns:
            print(f"Zielvariable '{target}' nicht in y_test_data. Überspringe.")
            continue
        y_test = y_test_data[target].copy()
        y_test_str = data['y_test'][target].astype(str)
        y_test_str = y_test_str.str.replace(",", ".")
        y_test = y_test_str.astype(float)

        # e) X_test nach den im Modell verwendeten Features filtern
        X_test_sub = X_test[selected_features].copy()
        
        # f) Vorhersage auf dem Testset
        y_test_pred = best_model.predict(X_test_sub)
        
        # g) Kennzahlen berechnen
        test_r2 = r2_score(y_test, y_test_pred)
        test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
        test_mae = mean_absolute_error(y_test, y_test_pred)
        
        # h) Vergleich mit Train/Val aus model_info
        train_r2 = model_info.get("train_r2", None)
        val_r2   = model_info.get("val_r2", None)
        train_rmse = model_info.get("train_rmse", None)
        val_rmse   = model_info.get("val_rmse", None)
        train_mae  = model_info.get("train_mae", None)
        val_mae    = model_info.get("val_mae", None)
        
        # i) Speichere Zusammenfassung in Dict
        results_list.append({
            "Zielvariable": target,
            "Train R²": train_r2,
            "Val R²": val_r2,
            "Test R²": round(test_r2, 4),
            "Train RMSE": train_rmse,
            "Val RMSE": val_rmse,
            "Test RMSE": round(test_rmse, 2),
            "Train MAE": train_mae,
            "Val MAE": val_mae,
            "Test MAE": round(test_mae, 2),
            "Features": selected_features
        })
    
    # 4) Ausgabe in schöner Tabellenform
    if results_list:
        results_df = pd.DataFrame(results_list)
        print("\n=== Zusammenfassung aller Modelle (Train/Val/Test) ===")
        print(results_df.to_string(index=False))
    else:
        print("Keine Ergebnisse. Wurden die JSON- und pkl-Dateien gefunden?")
        

if __name__ == "__main__":
    main()

# Gesamtanalyse der Regressionsmodelle

## 1. Modellperformance im Überblick

| Zielvariable | Set | R² | RMSE | MAE |
|--------------|-----|-----|------|-----|
| **Wiedergabedauer** | Train | 0.8642 | 35.03 | 22.38 |
| | Val | 0.6946 | 43.76 | 22.27 |
| | Test | 0.8471 | 33.62 | 23.00 |
| **Klickrate** | Train | 0.0438 | 2.29 | 1.77 |
| | Val | 0.0443 | 2.15 | 1.75 |
| | Test | 0.0216 | 2.21 | 1.75 |
| **Aufrufe** | Train | 0.0478 | 90393.92 | 44625.83 |
| | Val | 0.0399 | 133565.28 | 50106.51 |
| | Test | 0.0733 | 79979.54 | 44775.76 |


<img src="../references/images/Last1.JPG" width="600" alt="Modelperformance nach Kategorie"/>
<img src="../references/images/Last2.JPG" width="600" alt="Modelperformance Vergleich"/>
<img src="../references/images/Last3.JPG" width="600" alt="Anpassung Forward Selection"/>

#### Code für Grafikanzeige Modell-Performance Parameter

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting Style
plt.style.use('default')
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

# Feature Importance Daten
feature_importance = pd.DataFrame({
    'feature': ['theme_Bilder', 'theme_Krieg', 'theme_Sonstiges', 'Gestaltung_Thumbnail', 
                'Bewertung_Titel', 'theme_Wirtschaft', 'video_length_seconds', 
                'publish_weekday', 'publish_hour'],
    'original': [-12.072, 10.707, -6.316, 3.709, 0.578, 0.553, 0.205, -0.038, 0.034],
    'selected': [-13.146, 7.968, -9.667, 4.944, 2.269, -2.529, 0.243, 0.415, -0.063]
})

# Sortieren nach absolutem Einfluss der selected Werte
feature_importance = feature_importance.assign(
    abs_selected=lambda x: abs(x['selected'])
).sort_values('abs_selected', ascending=True)

# Performance Metriken
performance_metrics = pd.DataFrame({
    'metric': ['Wiedergabedauer', 'Klickrate', 'Aufrufe'],
    'train_r2': [0.8642, 0.0438, 0.0478],
    'val_r2': [0.6946, 0.0443, 0.0399],
    'test_r2': [0.8471, 0.0216, 0.0733]
})

# Forward Selection Progress
forward_progress = pd.DataFrame({
    'step': range(1, 10),
    'feature': ['video_length_seconds', 'theme_Sonstiges', 'theme_Krieg', 'theme_Bilder',
                'Bewertung_Titel', 'Gestaltung_Thumbnail', 'publish_hour', 
                'publish_weekday', 'theme_Wirtschaft'],
    'train_r2': [0.2, 0.35, 0.48, 0.62, 0.71, 0.78, 0.82, 0.85, 0.86],
    'val_r2': [0.19, 0.33, 0.45, 0.58, 0.64, 0.66, 0.68, 0.69, 0.69]
})

# Plotting
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 20))

# 1. Feature Importance Vergleich
feature_importance.plot(kind='barh', x='feature', y=['original', 'selected'], ax=ax1)
ax1.set_title('Feature Importance Vergleich')
ax1.set_xlabel('Koeffizient')
ax1.set_ylabel('Feature')
ax1.legend(['Original Modell', 'Forward Selection'])

# 2. R² Performance Vergleich
performance_metrics.plot(kind='bar', x='metric', y=['train_r2', 'val_r2', 'test_r2'], ax=ax2)
ax2.set_title('R² Performance Vergleich')
ax2.set_xlabel('Metrik')
ax2.set_ylabel('R²')
ax2.legend(['Train R²', 'Validation R²', 'Test R²'])

# 3. Forward Selection Progress
forward_progress.plot(x='step', y=['train_r2', 'val_r2'], marker='o', ax=ax3)
ax3.set_title('Forward Selection Fortschritt')
ax3.set_xlabel('Schritt')
ax3.set_ylabel('R²')
ax3.legend(['Train R²', 'Validation R²'])
ax3.grid(True)

plt.tight_layout()
plt.show()

# Optional: Speichern der Grafik
plt.savefig('regression_analysis_plots.png', dpi=300, bbox_inches='tight')


## 2. Zentrale Erkenntnisse

### Wiedergabedauer
- **Stärkste Performance**: R² zwischen 0.69 und 0.86
- Sehr stabile MAE-Werte (~22-23 Sekunden)
- Gute Generalisierung auf Testdaten (Test R² = 0.8471)

### Klickrate
- **Schwache Performance**: R² < 0.05 über alle Sets
- Konsistente MAE-Werte (~1.75%)
- Geringe Vorhersagekraft des Modells

### Aufrufe
- **Schwächste Performance**: R² < 0.08
- Hohe RMSE-Werte (~80.000-130.000 Aufrufe)
- Keine verlässliche Vorhersagekraft

## 3. Praktische Implikationen

### Für Content-Strategie
1. **Wiedergabedauer**
   - Gut vorhersagbar und steuerbar
   - Fokus auf identifizierte Einflussfaktoren sinnvoll
   - Verlässliche Basis für strategische Entscheidungen

2. **Klickrate**
   - Kaum durch betrachtete Faktoren beeinflussbar
   - Vermutlich stärkerer Einfluss externer Faktoren
   - Neue Prädiktoren oder alternative Modelle nötig

3. **Aufrufe**
   - Sehr schwer vorhersagbar
   - Wahrscheinlich stark von viralen Effekten abhängig
   - Fokus auf qualitative statt quantitative Optimierung

### Für Datenanalyse
1. **Modellierung**
   - Nichtlineare Ansätze für Klickrate und Aufrufe prüfen
   - Feature Engineering für bessere Prädiktion
   - Zeitreihenaspekte berücksichtigen

2. **Datenerhebung**
   - Zusätzliche Features für Klickrate und Aufrufe sammeln
   - Externe Faktoren (News, Trends) einbeziehen
   - Interaktionseffekte untersuchen

## 4. Empfehlungen

### Kurzfristig
1. Fokus auf Optimierung der Wiedergabedauer
2. Kritische Überprüfung der Thumbnail- und Titelstrategie
3. Realistische Erwartungen an Klickraten- und Aufrufvorhersagen

### Mittelfristig
1. Alternative Modellierungsansätze für Klickrate und Aufrufe
2. Erweiterung der Datenerfassung
3. A/B-Tests für Thumbnail- und Titelgestaltung

### Langfristig
1. Entwicklung eines hybriden Vorhersagemodells
2. Integration von Trend- und Konkurrenzdaten
3. Automatisierte Optimierungsstrategien

## 5. Fazit
Die Analyse zeigt deutlich, dass die Wiedergabedauer der am besten vorhersagbare und steuerbare Erfolgsfaktor ist. Klickraten und Aufrufe scheinen von komplexeren, möglicherweise externen Faktoren abzuhängen, die durch die aktuellen Modelle nicht ausreichend erfasst werden. Dies legt nahe, den Fokus der Content-Optimierung primär auf die Wiedergabedauer zu legen, während für Klickraten und Aufrufe alternative Analyse- und Optimierungsstrategien entwickelt werden sollten.